<img style="float:left;margin-right:1em;width:5em;" src="img/smelli.svg">

# `smelli` - the **SME**FT **L**ike**LI**hood

<div style="clear:left;"></div>

Lecture by Peter Stangl

- `smelli` articles: [arXiv:1810.07698](https://arxiv.org/abs/1810.07698), [arXiv:2012.12211](https://arxiv.org/abs/2012.12211)

- `smelli` API documentation: https://smelli.github.io/smelli/

- `smelli` source: https://github.com/smelli/smelli

- Slides of this lecture: https://github.com/peterstangl/flavio-lecture/tree/Ljubljana

# Motivation:
# Lessons learned from the $B$ anomalies

## The $B$ anomalies ($b\to s\,\ell\ell$ and $b\to c\tau\nu$)
<img style="float:left;width:15em;margin-right:2em;" src="img/P5p.svg"/>
<img style="float:left;width:15em;" src="img/BF_Kp_1403_8044_LHCb.svg"/>
<p style="clear:left;height:0em;margin:0em;"></p>
<img style="float:left;width:14em;margin-right:2em;margin-left:1em;" src="img/RK_2021.svg"/>
<img style="float:left;width:15em;margin-top:1em;" src="img/summer2023_preliminary_new.svg"/>
<p style="clear:left;height:0em;margin:0em;"></p>
<p style="text-align:left;font-size:60%;color:#91ac6b;width:65em;margin:0em;">
LHCb, arXiv:2003.04831, arXiv:2012.13241, arXiv:1403.8044, arXiv:1506.08777, arXiv:1606.04731, arXiv:1705.05802, arXiv:2103.11769, arXiv:2212.09153<br>HFLAV, hflav.web.cern.ch</p>

<div style="position:absolute;top:-14em;left:1em;">
<img style="width:14em;" src="img/RK_RKst_2022.svg"/>
</div>

## Model building - lessons learned

<ul style="float:left;width:28em;margin-top:0.2em;"><li>
    Model explaining $R_{D^{(*)}}$ using $b_L\to c_L\tau_L\nu_{\tau L}$
    <p style="margin-left:2em;">$b_L\to c_L\tau_L\nu_{\tau L}$ $\quad\xrightarrow{SU(2)_L}\quad$ $b_L\to s_L\nu_{\mu L}\nu_{\tau L}$</p>
        Constrained by $B\to K\nu\bar{\nu}$ searches
    <p style="text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;margin-top:0.5em;">
    Buras, Girrbach-Noe, Niehoff, Straub, arXiv:1409.4557</p>
</li></ul>
        <img style="float:left;margin-left:2em;margin-top:0.5em;height:6em;"; src="img/bctaunu_bsnunu.svg"/>

<ul style="float:left;width:27em;margin-top:0.2em;"><li>
    Model explaining $R_{D^{(*)}}$ and $R_{K^{(*)}}$ using mostly 3rd gen. couplings
 <br>
 Modifies LFU in $\tau$ and $Z$ decays, strongly constrained
    <p style="text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;margin-top:0.5em;">
    Feruglio, Paradisi, Pattori, arXiv:1705.00929</p>
</li></ul>
        <img style="float:left;margin-left:2em;margin-top:0em;height:7em;"; src="img/taulfu-penguin.svg"/>

<ul style="float:left;width:27em;margin-top:0.2em;"><li>
    Model explaining $b\to s\mu\mu$ using $tt\mu\mu$ interaction
 <br>
 Modifies $Z\to\mu\mu$, constrained by LEP
    <p style="text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;margin-top:0.5em;">
 Camargo-Molina, Celis, Faroughy, arXiv:1805.04917</p>
</li></ul>
        <img style="float:left;margin-left:1.5em;margin-top:0em;height:5em;"; src="img/diagrams-ttmumu.png"/>
        <p style="clear:left;"></p>

## What one would have to do

- Compute **all relevant observables** $\vec{O}$ (flavour,
  EWPO, ...) in terms of Lagrangian parameters $\vec\xi$
  <p style="margin-left:2em;">$\mathcal L_\text{NP}(\vec\xi) \to \vec{O}(\vec{\xi})$</p>

- Take into account loop / RGE effects
  <p style="margin-left:2em;">$\mathcal L_\text{NP}(\vec\xi) \xrightarrow{\Lambda_\text{NP}\to \Lambda_\text{IR}} \vec{O}(\vec{\xi})$</p>

- Compare to experiment
<p style="margin-left:2em;width:35em">$\vec{O}(\vec{\xi}) \to \underbrace{\mathcal L_\text{exp}(\vec{O}(\vec{\xi}))}_\text{Likelihood}$</p>

Tedious to do this for each model...

## The SMEFT approach

- Assuming $\Lambda_\text{NP}\gg v$, NP effects in flavour, EWPO, Higgs, top,... can be expressed in terms of Standard Model Effective Field Theory (SMEFT) Wilson coefficients
  <p style="margin-left:2em;">$\mathcal L_\text{SMEFT} = \mathcal L_\text{SM} + \sum_{n>4}\sum_i \frac{C_i}{\Lambda^{n-4}}O_i$</p>
  
  
<p style="text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;margin-top:-3.7em;">
 Buchmuller, Wyler, Nucl. Phys. B 268 (1986) 621
 <br>
 Grzadkowski, Iskrzynski, Misiak, Rosiek, arXiv:1008.4884</p>

- Powerful tool to connect model-building to phenomenology without
  need to recompute hundreds of observables in each model

  - Model building and matching:
    <p style="margin-left:2em;margin-bottom:1em;">$\mathcal L_\text{NP}(\vec\xi) \to \vec{C}(\vec\xi)\ \ \ \text{@}\ \ \Lambda_\text{NP}$</p>
  - *Model-independent* pheno:
    <p style="margin-left:2em;margin-bottom:1em;">$\vec{C}\xrightarrow{\Lambda_\text{NP}\to \Lambda_\text{IR}} \vec{O}(\vec{C}) \to \mathcal L_\text{exp}(\vec{O}(\vec{C}))$</p>
  - **SMEFT likelihood** $L_\text{exp}(\vec{C})$ can
    tremendously simplify analyses of NP models

# The global SMEFT likelihood

## The Global SMEFT likelihood

- **SMEFT likelihood function** $\mathcal L(\vec{C})$ can tremendously simplify analyses of NP models

- Several likelihood functions have been considered in the context of EFT fits
    <p style="margin-left:2em;">
    $\mathcal L(\vec C) = \mathcal L_\text{EW + Higgs}(\vec C_\text{EW + Higgs}) \times \ldots$<br>
    $\mathcal L(\vec C) = \mathcal L_\text{top physics}(\vec C_\text{top physics}) \times \ldots$<br>
    $\mathcal L(\vec C) = \mathcal L_\text{$B$ physics}(\vec C_\text{$B$ physics}) \times \ldots$<br>
    $\mathcal L(\vec C) = \mathcal L_\text{LFV}(\vec C_\text{LFV}) \times \ldots$
    </p>
    

<p style="text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;margin-top:-9em;">
cf. eg.
Falkowski, Mimouni, arXiv:1511.07434<br>
Falkowski, González-Alonso, Mimouni, arXiv:1706.03783<br>
Ellis, Murphy, Sanz, You, arXiv:1803.03252<br>
Biekötter, Corbett, Plehn, arXiv:1812.07587<br>
Hartland et al., arXiv:1901.05965<br>
Ellis, Madigan, Mimasu, Sanz, You, arXiv:2012.02779<br>
...
</p>

- But these likelihood functions should **not be considered separately** since RG (loop) effects mix different sectors and UV models match to several sectors

- We need to consider the **global SMEFT likelihood**

## Implementing the global SMEFT likelihood

-  Based on
<img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/flavio.svg">**flavio**,
<img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/wilson.png">**wilson**, and
<img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/wcxf.svg">**WCxf**,
we have started building the <br> global **SME**FT **L**ike**LI**hood <img style="display:inline;margin-left:0.1em;margin-right:0.3em;margin-bottom:0.55em;height:.9em;" src="img/smelli.svg">`smelli` https://github.com/smelli
  <p style="float:left;width:100%;margin:0em;text-align:right;font-size:60%;color:#91ac6b;padding-right:3em;padding-left:3em;margin-top:-3em;">Aebischer, Kumar, PS, Straub, arXiv:1810.07698
    <br>
    PS, arXiv:2012.12211
</p>
  <p style="clear:left;margin:0em;height:0em;padding:0em"></p>

<ul style="float:left;width:26em;margin-top:0.2em;"><li>
    $
  L(\vec C) \approx
  \prod_{i} L_\text{exp}^i(\vec{O}_\text{th}(\vec C, \vec \theta_0))
  \times
  \tilde{L}_\text{exp}(\vec{O}_\text{th}(\vec C, \vec \theta_0))
     $
    where
    <ul>
        <li>$\vec C$ WET or SMEFT Wilson coefficients</li>
        <li>$\vec \theta_0$ fixed nuisance parameters</li>
        <li>$\vec{O}_\text{th}(\vec C, \vec \theta_0)$ observable predictions</li>
        <li>$ L_\text{exp}^i(\vec{O})$ experimental likelihood from<br>measurement $i$ for observables $\vec{O}$</li>
        <li>$\tilde{L}_\text{exp}(\vec O)$ modified exp. likelihood:<br>$-2\ln \tilde{L}_\text{exp}(\vec O)=\vec D^T (\Sigma_\text{exp}+\Sigma_\text{th})^{-1} \vec D \,,$<br>with $\vec D = \vec O - \vec O_\text{exp}$ and
      covariance<br>matrices $\Sigma_\text{exp,th}$ (Gaussian approx.)</li>
    </ul>
    </li></ul>
        <img style="float:left;margin-left:2em;margin-top:1em;height:12em;"; src="img/sketch-new.svg"/>

# Using `smelli`

## Installing `smelli`

- Requirements:
    - working installation of **Python** version **3.7** or above
    - **pip** (Python package manager)

- Installation from the command line:

  **`python3 -m pip install smelli --user`**

  - downloads `smelli` with all dependencies from Python package archive (PyPI)
  - installs it in user's home directory (no need to be root)

## Instantiating the likelihood

In [ ]:
from smelli import GlobalLikelihood

In [ ]:
gl = GlobalLikelihood()

In [ ]:
gl.eft, gl.basis

In [ ]:
gl_wet = GlobalLikelihood(eft='WET', basis='flavio')
gl_wet.eft, gl_wet.basis

## Fixing a point in WC space: 3 equivalent ways

In [ ]:
pp = gl.parameter_point({'lq3_2223': 1e-9}, scale=1000)

In [ ]:
from wilson import Wilson
w = Wilson({'lq3_2223': 1e-9}, scale=1000, eft='SMEFT', basis='Warsaw')
pp = gl.parameter_point(w)

In [ ]:
! cat my_wcxf.yaml

In [ ]:
pp = gl.parameter_point('my_wcxf.yaml')

## Computing the likelihood

In [ ]:
%%time
pp.log_likelihood_dict()

In [ ]:
%%time
pp.log_likelihood_global()

In [ ]:
pp.chi2_dict()

In [ ]:
pp.pvalue_dict()

## Table of observables

In [ ]:
df = pp.obstable()

In [ ]:
df

In [ ]:
df.sort_values('pull SM')[:3]

In [ ]:
df.loc[['Rtaul(B->D*lnu)']]

## Plots

In [ ]:
import flavio.plots as fpl
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [ ]:
gl_ewpt_higgs = GlobalLikelihood(fix_ckm=True, include_likelihoods=['likelihood_ewpt.yaml', 'likelihood_higgs.yaml'])

In [ ]:
def wc_fct(S, T):
    return {
        'phiWB': S * 7.643950529889027e-08, # gL*gY/(16*pi*v**2)
        'phiD': -T * 2.5793722852276787e-07, # gL**2*gY**2/(2*pi*v**2)/(gL**2+gY**2)
    }

In [ ]:
%%time

plot_data = gl_ewpt_higgs.plot_data_2d(
    wc_fct,
    91.1876,
    -0.2, 0.2, -0.1, 0.3,
    steps=10,
    threads=8,
)

In [ ]:
levels_1sig = [fpl.delta_chi2(1, dof=2)]
levels_123sig = [fpl.delta_chi2(n_sigma, dof=2) for n_sigma in (1,2,3)]
interpolation_factor = 9

In [ ]:
plt.figure(figsize=(5,5))
fpl.contour(**plot_data['likelihood_higgs.yaml'], levels=levels_1sig, label=r"Higgs ($1\sigma$)",
            interpolation_factor=interpolation_factor, color='C0')
fpl.contour(**plot_data['likelihood_ewpt.yaml'], levels=levels_1sig, label=r"EWPT ($1\sigma$)",
            interpolation_factor=interpolation_factor, color='C1')
fpl.contour(**plot_data['global'], levels=levels_123sig, label=r"global",
            interpolation_factor=interpolation_factor, color='C3')
plt.axhline(c='0.6', linewidth=1)
plt.axvline(c='0.6', linewidth=1)
plt.xlabel(r'$S$')
plt.ylabel(r'$T$')
plt.legend()
plt.show()

## Outlook to `smelli` v3.0 (work in progress)

- Major speed improvement (orders of magnitude)
- High-mass Drell-Yan tails: $pp\to e\nu,\mu\nu$ and $pp\to e^+e^-,\mu^+\mu^-$ (already available in `flavio`)
- EDMs: neutron, atomic, and molecular (already available in `flavio`)

## `flavio` and `smelli` are open source projects

Contributions from the user community are very welcome and encouraged!

If you want to use `flavio` or `smelli` for your research and you
- implement a new observable
- add or update measurements
- have an idea for a new feature

please get in contact by opening an issue on GitHub:

- https://github.com/flav-io/flavio/
- https://github.com/smelli/smelli